In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# I due array con i parametri necessari per scorrere le varie pagine con le richieste di rete
# per le pagine basta un for 
anni = ["2014", "2015", "2016", "2017", "2018", "2019","2020","2021", "2022"]
italian_regions = [
    "Abruzzo",
    "Basilicata",
    "Calabria",
    "Campania",
    "Emilia-Romagna",
    "Friuli-Venezia Giulia",
    "Lazio",
    "Liguria",
    "Lombardia",
    "Marche",
    "Molise",
    "Piemonte",
    "Puglia",
    "Sardegna",
    "Sicilia",
    "Toscana",
    "Trentino-Alto Adige",
    "Umbria",
    "Valle d'Aosta",
    "Veneto"
]
semestri = ["semestre_i", "semestre_ii"]

In [3]:
'''
metodo per fare web scraping dal sito misurainternet.it dell'agcom 
e parsa in un dataframe

@params year string anno di ricerca 
@params page string  pagina della tabella 
@params period string sembra che dando il semestre si ottengano più dati
@return dataset_speed Dataframe
//TODO il token per il middleware e sembra non scadere, nel caso si dinamizza anche quello

'''
def getDataset(year, page, period="anno"):
    url = f'https://misurainternet.it/valori_statistici/?csrfmiddlewaretoken=PmowMH9VasBwM58EEVdl9W1ARGSIvgCyYTL0uODEECL2GxQn952STNPYUtN6rNqQ&operatore=&regione=&anno={year}&periodo={period}&page={page}'
    list_page_event = requests.get(url)
    site = BeautifulSoup(list_page_event.text, "html.parser")
    headers = [header.text for header in site.find_all("th")]
    table_clean = [row.text.replace(" ","") for row in site.find_all("td")]
    table_clean_regroup = [table_clean[i:i+12] for i in range(0, len(table_clean), 12)]
    dataset_speed = pd.DataFrame(table_clean_regroup, columns=headers)
    return dataset_speed


In [4]:
dataset_speed= pd.DataFrame()
for anno in anni:
    sem_1 = pd.DataFrame()
    sem_2 = pd.DataFrame()
    annualita = pd.DataFrame()
    for index, (semestre) in enumerate(semestri):
        for page in range(1,10):
            try:
                if index == 0: 
                    sem_1 = pd.concat([sem_1,getDataset(anno,page,semestre)],ignore_index=True) 
                else:
                    sem_2 = pd.concat([sem_2,getDataset(anno,page,semestre)],ignore_index=True)
            except:
                break
        annualita = pd.concat([sem_1,sem_2], ignore_index=True)
        annualita["Anno"]=anno
    dataset_speed = pd.concat([dataset_speed,annualita], ignore_index=True)
        

In [5]:
dataset_speed

,Operatore,Profilo,Velocità (down/up),Territorio,Periodo,Tipo,Media,Dev. Std.,Min,Max,TP%,Misure,Anno
0,Acantho,BigBand,"\n7,0Mbps512.0kbps\n",Nazionale,\nGen2014Giu2014\n,Download,"5,4Mbps","969,4","3,3Mbps","6,2Mbps",,43900,2014
1,Acantho,BigBand,"\n7,0Mbps512.0kbps\n",Nazionale,\nGen2014Giu2014\n,Ping,53ms,"4,9",,\n61ms\n,0%,21946,2014
2,Acantho,BigBand,"\n7,0Mbps512.0kbps\n",Nazionale,\nGen2014Giu2014\n,Upload,487.0kbps,"3,9",484.0kbps,488.0kbps,,43888,2014
3,Acantho,BigBand,"\n20,0Mbps1,0Mbps\n",Nazionale,\nGen2014Giu2014\n,Download,"13,2Mbps","1398,1","10,5Mbps","15,0Mbps",,43860,2014
4,Acantho,BigBand,"\n20,0Mbps1,0Mbps\n",Nazionale,\nGen2014Giu2014\n,Ping,33ms,"5,2",,\n41ms\n,0%,21920,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...
941,Wind,"WindHomeADSL,3ADSL,AbsoluteADSL,AllInclusive,A...","\n12,0Mbps1,0Mbps\n",Nazionale,\nLug2022Dic2022\n,Download,"13,6Mbps","6543,8","11,8Mbps",,,8505,2022
942,Wind,"WindHomeADSL,3ADSL,AbsoluteADSL,AllInclusive,A...","\n14,0Mbps1,0Mbps\n",Nazionale,\nLug2022Dic2022\n,Download,"11,6Mbps","820,3","10,8Mbps",,,18045,2022
943,Wind,"WindHomeADSL,3ADSL,AbsoluteADSL,AllInclusive,A...","\n17,0Mbps1,0Mbps\n",Nazionale,\nLug2022Dic2022\n,Download,"15,0Mbps","31,7","15,0Mbps",,,9020,2022
944,Wind,"WindHomeADSL,3ADSL,AbsoluteADSL,AllInclusive,A...","\n20,0Mbps1,0Mbps\n",Nazionale,\nLug2022Dic2022\n,Download,"17,6Mbps","2327,6","16,2Mbps",,,33352,2022


In [43]:
df_download_media = dataset_speed[(dataset_speed.Tipo == "Download")].reset_index()

In [ ]:
È

In [51]:
import re
m = []
for i in range(0, len(df_download_media)):
    m.append(float(re.findall("[-+]?(?:\d*\,*\d+)", df_download_media.iloc[i]["Media"])[0].replace(',','.')))
df_download_media["media_f"] = m
df_download_media.info()
df_download_media_regroup = df_download_media.groupby(["media_f","Operatore"]).mean()
df_download_media_regroup


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 637 entries, 0 to 636
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               637 non-null    int64  
 1   Operatore           637 non-null    object 
 2   Profilo             637 non-null    object 
 3   Velocità (down/up)  637 non-null    object 
 4   Territorio          637 non-null    object 
 5   Periodo             637 non-null    object 
 6   Tipo                637 non-null    object 
 7   Media               637 non-null    object 
 8   Dev. Std.           637 non-null    object 
 9   Min                 637 non-null    object 
 10  Max                 637 non-null    object 
 11  TP%                 637 non-null    object 
 12  Misure              637 non-null    object 
 13  Anno                637 non-null    object 
 14  media_f             637 non-null    float64
dtypes: float64(1), int64(1), object(13)
memory usage: 74.8+ K

TypeError: agg function failed [how->mean,dtype->object]